In [ ]:
#|hide
#|default_exp read

# read
> Reading a notebook, and initial bootstrapping for notebook exporting

In [ ]:
#|export
from datetime import datetime
from fastcore.imports import *
from fastcore.foundation import *
from fastcore.basics import *
from fastcore.imports import *
from fastcore.meta import *
from fastcore.script import *
from fastcore.xdg import *
from fastcore.xtras import *

import ast,functools,yaml
from IPython.display import Markdown
from configparser import ConfigParser
from execnb.nbio import read_nb, NbCell
from pprint import pformat,pprint

In [ ]:
#|hide
from fastcore.test import *
import tempfile

In [ ]:
#|export
def yaml_str(s:str):
    "Create a valid YAML string from `s`"
    if s[0]=='"' and s[-1]=='"': return s
    res = s.replace('\\', '\\\\').replace('"', r'\"')
    return f'"{res}"'

In [ ]:
#|export
_re_title = re.compile(r'^#\s+(.*)[\n\r]+(?:^>\s+(.*))?', flags=re.MULTILINE)
_re_fm = re.compile(r'^---(.*\S+.*)---', flags=re.DOTALL)
_re_defaultexp = re.compile(r'^\s*#\|\s*default_exp\s+(\S+)', flags=re.MULTILINE)

def dict2fm(d): return '---\n'+yaml.dump(d)+'\n---'
def _cellsrc(c): return c.get('source', None) or ''
def _celltyp(nb, cell_type): return L(nb.cells).filter(lambda c: c.cell_type == cell_type)
def _select_cell(nb, cell_type, f): return first(_celltyp(nb, cell_type).filter(lambda c: f(_cellsrc(c))))

In [ ]:
#|export
def default_exp(nb):
    "get the default_exp from a notebook"
    cell = _select_cell(nb, 'code', _re_defaultexp.search)
    if cell: 
        exp = _re_defaultexp.search(_cellsrc(cell))
        return exp.group(1) if exp else None

In [ ]:
#|hide
_testnb = read_nb('../tests/docs_test.ipynb')
test_eq(default_exp(_testnb), 'foobar')
_testnb = read_nb('../tests/2020-09-01-fastcore.ipynb')
test_eq(default_exp(_testnb), None)

In [ ]:
#|export
def yml2dict(s:str, rm_fence=True):
    "convert a string that is in a yaml format to a dict"
    if not s: return {}
    if rm_fence: 
        match = _re_fm.search(s.strip())
        if match: s = match.group(1)
    return yaml.safe_load(s)

In [ ]:
#|hide
_yml = """
---
foo: bar
comments:
  hypothesis: 
    theme: clean
categories: [c1, c2]
---
"""

test_eq(yml2dict(_yml), {'foo': 'bar', 'comments': {'hypothesis': {'theme': 'clean'}}, 'categories': ['c1', 'c2']})

In [ ]:
#|export
class NB:
    "Notebook with tools for manipulating front matter"
    def __init__(self, 
                 nb # an AttrDict or Path to a notebook
                ): 
        if isinstance(nb, NB): self.nb = nb.nb
        elif isinstance(nb, (str, Path)): self.nb = read_nb(nb)
        else: self.nb=nb
        self._raw_fm_dict = yml2dict(getattr(self._fm_cell, 'source', None))
        
    def __getattr__(self, attr): return getattr(self.nb, attr)

    def __getitem__(self, x): return getattr(self, x)

    @property
    def default_exp(self): return default_exp(self.nb)

    @property
    def text(self): return concat(L(self.nb.cells).attrgot('source').filter())
    
    def print_txt(self, n=None): 
        for t in self.text[:n]: print(t + '\n')

    @property
    def title_cell(self): return _select_cell(self.nb, 'markdown', _re_title.search)        
                                       
    @property
    def _fm_cell(self): return _select_cell(self.nb, 'raw', _re_fm.search)
             
    @property
    def raw_fm_dict(self): return self._raw_fm_dict
                                       
    @raw_fm_dict.setter
    def raw_fm_dict(self, val):
        if not val: return
        if self._fm_cell: self.cells.remove(self._fm_cell)
        self.nb.cells.insert(0, NbCell(0, dict(cell_type='raw', metadata={}, source=dict2fm(val), directives_={})))
        self._raw_fm_dict = val
        
    def update_raw_fm(self, fmdict): self.raw_fm_dict = merge(self.raw_fm_dict, fmdict)
        
    @property
    def _md_fm_dict(self): 
        "Infer the front matter from a notebook's markdown formatting"
        if not self.title_cell: return {}
        title_match = _re_title.match(self.title_cell.source or '')
        if title_match:
            title,desc=title_match.groups()
            flags = re.findall('^-\s+(.*)', self.title_cell.source, flags=re.MULTILINE)
            flags = [s.split(':', 1) for s in flags if ':' in s] if flags else []
            flags = merge({k:v for k,v in flags if k and v}, 
                          {'title':yaml_str(title)}, {'description':yaml_str(desc)} if desc else {})
            return yml2dict('\n'.join([f"{k}: {flags[k]}" for k in flags]))
        else: return {}
    
    @property
    def fmdict(self): return merge(self._md_fm_dict, self.raw_fm_dict)

You can access the raw front matter with `NB.raw_fm_dict`:

In [ ]:
## A notebook that has markdown frontmatter
_nb = NB('../tests/2020-02-20-test.ipynb')
assert not _nb.raw_fm_dict 

In [ ]:
#|hide
assert '# Fastpages Notebook Blog Post' in _nb.title_cell.source

In [ ]:
# A notebook that has raw frontmatter
_nb = NB('../tests/docs_test.ipynb')
test_eq(_nb.raw_fm_dict, {'execute': {'echo': False}})

The exported library you set with `#|default_exp` is also available in the `default_exp` property:

In [ ]:
assert _nb.default_exp == 'foobar'

If you update `raw_fm_dict` it will override the notebook's raw frontmatter as well. Consider the below notebook with the following front matter:

In [ ]:
_nb.print_txt(1)

---
execute:
  echo: false
---



We can update `raw_fm_dict` and it will update the YAML as well:

In [ ]:
_nb.raw_fm_dict = merge(_nb.raw_fm_dict, {'author': 'Hamel Husain'})
_nb.print_txt(1)

---
author: Hamel Husain
execute:
  echo: false

---



We can also use `update_raw_fm` to update the YAML:

In [ ]:
_nb.update_raw_fm({'author': 'Hamel Husain'})
_nb.print_txt(1)

---
author: Hamel Husain
execute:
  echo: false

---



In [ ]:
#|hide
assert 'author: Hamel Husain' in _nb.text[0]

In addition to raw front matter, Notebooks can have Markdown front matter.  Below is the cell with markdown front matter. 

In [ ]:
print(_nb.title_cell.source)

# a title
> A description
- key1: value1
- key2: value2
- categories: [c1, c2]


We can access the combined markdown and raw front matter with the `fmdict` property:

In [ ]:
_nb.fmdict

{'key1': 'value1',
 'key2': 'value2',
 'categories': ['c1', 'c2'],
 'title': 'a title',
 'description': 'A description',
 'execute': {'echo': False},
 'author': 'Hamel Husain'}

In [ ]:
#|hide
test_eq(_nb.fmdict,
{'key1': 'value1',
 'key2': 'value2',
 'categories': ['c1', 'c2'],
 'title': 'a title',
 'description': 'A description',
 'execute': {'echo': False},
 'author': 'Hamel Husain'})

Any conflicts between the markdown and raw front matter will be resolved in favor of the raw front matter:

In [ ]:
_nb.update_raw_fm({'title': 'A New Title'})
assert 'A New Title' in _nb.fmdict['title']
_nb.print_txt(1)

---
author: Hamel Husain
execute:
  echo: false
title: A New Title

---



In [ ]:
#|export
@patch
def repl_fm(self:NB, fmdict:dict):
    "replace raw front matter with a new dictionary `fmdict` and remove markdown frontmatter"
    self.raw_fm_dict = fmdict
    if self.title_cell: self.cells.remove(self.title_cell)

In [ ]:
_nb = NB('../tests/docs_test.ipynb')
_nb.repl_fm({'title': 'new title', 'author': 'new author'})
test_eq(_nb.text[0], '---\nauthor: new author\ntitle: new title\n\n---')

## Helpers

In [ ]:
#|export
def create_output(txt, mime):
    "Add a cell output containing `txt` of the `mime` text MIME sub-type"
    return [{"data": { f"text/{mime}": str(txt).splitlines(True) },
             "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]

In [ ]:
#|export
def show_src(src, lang='python'): return Markdown(f'```{lang}\n{src}\n```')

In [ ]:
show_src("print(create_output('text', 'text/plain'))")

```python
print(create_output('text', 'text/plain'))
```

## Config

nbdev allows per-user and per-repo settings files in the `ConfigParser` format, conveniently read and written using fastcore's `Config` class. Settings are searched in the following order:

- Default settings: see `nbdev_create_config` for a full reference of possible settings and their defaults
- Repo settings: `settings.ini` file in the root of each project
- User settings: `settings.ini` file following the [XDG base directory specification](https://fastcore.fast.ai/xdg.html), by default: `~/.config/nbdev/settings.ini`

In [ ]:
#|export
_nbdev_home_dir = 'nbdev' # sub-directory of xdg base dir
_nbdev_config_name = 'settings.ini'

In [ ]:
#|exporti
def apply_defaults(
    cfg,
    lib_name:str=None, # Package name, defaults to local repo folder name
    branch='master', # Repo default branch
    git_url='https://github.com/%(user)s/%(lib_name)s', # Repo URL
    custom_sidebar:bool_arg=False, # Create custom sidebar?
    nbs_path='.', # Path to notebooks
    lib_path='%(lib_name)s', # Path to package root
    doc_path='_docs', # Path to rendered docs
    tst_flags='', # Test flags
    version='0.0.1', # Version of this release
    doc_host='https://%(user)s.github.io',  # Hostname for docs
    doc_baseurl='/%(lib_name)s',  # Base URL for docs
    keywords='nbdev jupyter notebook python', # Package keywords
    license='apache2', # License for the package
    copyright:str=None, # Copyright for the package, defaults to '`current_year` onwards, `author`'
    status='3', # Development status PyPI classifier
    min_python='3.7', # Minimum Python version PyPI classifier
    audience='Developers', # Intended audience PyPI classifier
    language='English', # Language PyPI classifier
    recursive:bool_arg=False, # Include subfolders in notebook globs?
    black_formatting:bool_arg=False, # Format libraries with black?
    readme_nb='index.ipynb', # Notebook to export as repo readme
    title='%(lib_name)s', # Quarto website title
    allowed_metadata_keys='', # Preserve the list of keys in the main notebook metadata
    allowed_cell_metadata_keys='', # Preserve the list of keys in cell level metadata
    jupyter_hooks=True, # Run Jupyter hooks?
    clean_ids=True, # Remove ids from plaintext reprs?
):
    "Apply default settings where missing in `cfg`"
    if lib_name is None:
        _parent = Path.cwd().parent
        lib_name = _parent.parent.name if _parent.name=='nbs' else _parent.name
    if copyright is None and hasattr(cfg,'author'): copyright = f"{datetime.now().year} ownwards, {cfg.author}"
    for k,v in locals().items():
        if not (k.startswith('_') or k=='cfg' or k in cfg): cfg[k] = v
    return cfg

In [ ]:
#|export
@call_parse
@delegates(apply_defaults, but='cfg')
def nbdev_create_config(
    user:str, # Repo username
    author:str, # Package author's name
    author_email:str, # Package author's email address
    description:str, # Short summary of the package
    path:str='.', # Path to create config file
    cfg_name:str=_nbdev_config_name, # Name of config file to create
    **kwargs
):
    "Create a config file"
    d = {k:v for k,v in locals().items() if k not in ('path','cfg_name')}
    cfg = Config(path, cfg_name, d, save=False)
    cfg = apply_defaults(cfg, **kwargs)
    cfg.save()

This is a wrapper for `fastcore`'s `save_config_file` with nbdev's required settings. It is also installed as a CLI command. The table above also serves as a full reference of nbdev's settings (excluding the `path` and `cfg_name` parameters which decide where the config file is saved). For more about PyPI classifiers, see [_Classifiers_](https://pypi.org/classifiers/).

In [ ]:
#|export
def _nbdev_config_file(cfg_name=_nbdev_config_name, path=None):
    cfg_path = path = Path.cwd() if path is None else Path(path)
    while cfg_path != cfg_path.parent and not (cfg_path/cfg_name).exists(): cfg_path = cfg_path.parent
    if not (cfg_path/cfg_name).exists(): cfg_path = path
    return cfg_path/cfg_name

In [ ]:
test_eq(_nbdev_config_file(), Path.cwd().parent/'settings.ini')

In [ ]:
#|export
def _xdg_config_paths(cfg_name=_nbdev_config_name):
    xdg_config_paths = reversed([xdg_config_home()]+xdg_config_dirs())
    return [o/_nbdev_home_dir/cfg_name for o in xdg_config_paths]

In [ ]:
#|export
@functools.lru_cache(maxsize=None)
def get_config(cfg_name=_nbdev_config_name, path=None):
    "`Config` for ini file found in `path` (defaults to `cwd`)"
    cfg_file = _nbdev_config_file(cfg_name, path)
    extra_files = _xdg_config_paths(cfg_name)
    cfg = Config(cfg_file.parent, cfg_file.name, extra_files=extra_files)
    return apply_defaults(cfg)

`get_config` searches for repo `settings.ini` in the current directory, and then in all parent directories, stopping when it is found. Default values for optional settings are applied to the resulting `Config`, see `nbdev_create_config` for a full reference of nbdev's settings.

In [ ]:
cfg_name = 'test_settings.ini'
nbdev_create_config('fastai','author','author@fast.ai','description','..',cfg_name)
cfg = get_config(cfg_name)
test_eq(cfg.lib_name, 'nbdev')
test_eq(cfg.git_url, 'https://github.com/fastai/nbdev')
cwd = Path.cwd()
test_eq(cfg.config_path, cwd.parent.absolute())
test_eq(cfg.path('lib_path'), cwd.parent/'nbdev')
test_eq(cfg.path('nbs_path'), cwd.parent)
test_eq(cfg.path('doc_path'), cwd.parent/'_docs')
cfg.config_file.unlink()
get_config.cache_clear()

In [ ]:
#|hide
# `get_config` returns defaults if no config file exists
with tempfile.TemporaryDirectory() as d, working_directory(d): print(get_config())
get_config.cache_clear()

{'lib_name': 'T', 'branch': 'master', 'git_url': 'https://github.com/%(user)s/%(lib_name)s', 'custom_sidebar': 'False', 'nbs_path': '.', 'lib_path': '%(lib_name)s', 'doc_path': '_docs', 'tst_flags': '', 'version': '0.0.1', 'doc_host': 'https://%(user)s.github.io', 'doc_baseurl': '/%(lib_name)s', 'keywords': 'nbdev jupyter notebook python', 'license': 'apache2', 'copyright': 'None', 'status': '3', 'min_python': '3.7', 'audience': 'Developers', 'language': 'English', 'recursive': 'False', 'black_formatting': 'False', 'readme_nb': 'index.ipynb', 'title': '%(lib_name)s', 'allowed_metadata_keys': '', 'allowed_cell_metadata_keys': '', 'jupyter_hooks': 'True', 'clean_ids': 'True'}


In [ ]:
#|export
def config_key(c, default=None, path=True, missing_ok=None):
    "Look for key `c` in settings.ini and fail gracefully if not found and no default provided"
    if missing_ok is not None:
        warn("`missing_ok` is no longer used. Don't pass it to `config_key` to silence this warning.")
    cfg = get_config()
    res = cfg.path(c, default) if path else cfg.get(c, default)
    if res is None: raise ValueError(f'`{c}` not specified in {_nbdev_config_name}')
    return res

In [ ]:
#|hide
# `config_key` returns defaults if no config file exists
with tempfile.TemporaryDirectory() as d, working_directory(d): print(config_key('lib_path', path=False))
get_config.cache_clear()

T


##  Exporting a basic module

In [ ]:
#|export
_init = '__init__.py'

def _has_py(fs): return any(1 for f in fs if f.endswith('.py'))

def add_init(path):
    "Add `__init__.py` in all subdirs of `path` containing python files if it's not there already"
    # we add the lowest-level `__init__.py` files first, which ensures _has_py succeeds for parent modules
    path = Path(path)
    path.mkdir(exist_ok=True)
    if not (path/_init).exists(): (path/_init).touch()
    for r,ds,fs in os.walk(path, topdown=False):
        r = Path(r)
        subds = (os.listdir(r/d) for d in ds)
        if _has_py(fs) or any(filter(_has_py, subds)) and not (r/_init).exists(): (r/_init).touch()

Python modules require a `__init.py__` file in all directories that are modules. We assume that all directories containing a python file (including in subdirectories of any depth) is a module, and therefore add a `__init__.py` to each.

In [ ]:
with tempfile.TemporaryDirectory() as d:
    d = Path(d)
    (d/'a/b').mkdir(parents=True)
    (d/'a/b/f.py').touch()
    (d/'a/c').mkdir()
    add_init(d)
    assert not (d/'a/c'/_init).exists(), "Should not add init to dir without py file"
    for e in [d, d/'a', d/'a/b']: assert (e/_init).exists(),f"Missing init in {e}"

## Export -

In [ ]:
#|export
def write_cells(cells, hdr, file, offset=0):
    "Write `cells` to `file` along with header `hdr` starting at index `offset` (mainly for nbdev internal use)"
    for cell in cells:
        if cell.source.strip(): file.write(f'\n\n{hdr} {cell.idx_+offset}\n{cell.source}')

In [ ]:
#|export
def basic_export_nb(fname, name, dest=None):
    "Basic exporter to bootstrap nbdev"
    if dest is None: dest = config_key('lib_path')
    fname,dest = Path(fname),Path(dest)
    nb = read_nb(fname)

    # grab the source from all the cells that have an `export` comment
    cells = L(cell for cell in nb.cells if re.match(r'#\s*\|export', cell.source))

    # find all the exported functions, to create `__all__`:
    trees = cells.map(NbCell.parsed_).concat()
    funcs = trees.filter(risinstance((ast.FunctionDef,ast.ClassDef))).attrgot('name')
    exp_funcs = [f for f in funcs if f[0]!='_']

    # write out the file
    with (dest/name).open('w') as f:
        f.write(f"# %% auto 0\n__all__ = {exp_funcs}")
        write_cells(cells, f"# %% {fname.relpath(dest)}", f)
        f.write('\n')

This is a simple exporter with just enough functionality to correctly export this notebook, in order to bootstrap the creation of nbdev itself.

In [ ]:
#|hide
#| eval: false
path = Path('../nbdev')
(path/'read.py').unlink(missing_ok=True)

add_init(path)
basic_export_nb("01_read.ipynb", 'read.py')

g = exec_new('from nbdev import read')
assert g['read'].add_init
assert 'add_init' in g['read'].__all__